# ECON 0150 | Replication Notebook

**Title:** Minimum Wage and Voter Turnout

**Original Authors:** Yee

**Original Date:** Fall 2024

---

This notebook replicates the analysis from a student final project in ECON 0150: Economic Data Analysis.

## About This Replication

**Research Question:** Is there a relationship between minimum wage and voter turnout?

**Data Source:** State-level election turnout data (1984-2024) with minimum wage and median household income

**Methods:** Multiple regression: Voter_Turnout ~ Minimum_Wage + Median_Household_Income

**Main Finding:** No significant relationship between minimum wage and voter turnout (coef = 0.15, p = 0.38). R² = 0.008.

**Course Concepts Used:**
- Multiple regression with controls
- Scatter plots with regression lines
- Partial regression plots
- Panel data across time

---
## Step 0 | Setup

In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [ ]:
# Load data from course website
base_url = 'https://tayweid.github.io/econ-0150/projects/replication/replications/0053/data/'

data = pd.read_csv(base_url + 'cleaned_election_turnout.csv')

print(f"Number of observations: {len(data)}")
data.head()

---
## Step 1 | Data Preparation

In [ ]:
# Check columns
print("Columns:", data.columns.tolist())

In [ ]:
# Clean data
# Strip whitespace from column names
data.columns = data.columns.str.strip()

# Clean Median_Household_Income: remove $ and commas
if 'Median_Household_Income' in data.columns:
    data['Median_Household_Income'] = data['Median_Household_Income'].astype(str).str.replace('$', '', regex=False).str.replace(',', '', regex=False)
    data['Median_Household_Income'] = pd.to_numeric(data['Median_Household_Income'], errors='coerce')

# Drop missing values
data = data.dropna(subset=['Minimum_Wage', 'Voter_Turnout', 'Median_Household_Income'])

print(f"\nCleaned data: {len(data)} observations")
data.head()

---
## Step 2 | Data Exploration

In [ ]:
# Summary statistics
print("Summary Statistics:")
print(data[['Minimum_Wage', 'Voter_Turnout', 'Median_Household_Income']].describe())

In [ ]:
# Correlation
correlation = data['Minimum_Wage'].corr(data['Voter_Turnout'])
print(f"\nCorrelation between minimum wage and voter turnout: {correlation:.3f}")

---
## Step 3 | Visualization

In [ ]:
# Scatter plot with regression line
plt.figure(figsize=(10, 6))
sns.regplot(x='Minimum_Wage', y='Voter_Turnout', data=data, scatter_kws={'alpha': 0.6})
plt.title('Minimum Wage vs Voter Turnout')
plt.xlabel('State Minimum Wage ($)')
plt.ylabel('Voter Turnout (%)')
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# Partial regression plot (controlling for income)
# Residualize both variables on income
X_income = sm.add_constant(data['Median_Household_Income'])

# Residual turnout
model_turnout = sm.OLS(data['Voter_Turnout'], X_income).fit()
data['Turnout_resid'] = model_turnout.resid

# Residual minimum wage
model_mw = sm.OLS(data['Minimum_Wage'], X_income).fit()
data['MW_resid'] = model_mw.resid

plt.figure(figsize=(10, 6))
sns.regplot(x='MW_resid', y='Turnout_resid', data=data, scatter_kws={'alpha': 0.6})
plt.title('Partial Regression Plot:\nMinimum Wage vs Voter Turnout (Controlling for Median Income)')
plt.xlabel('Minimum Wage (residualized)')
plt.ylabel('Voter Turnout (residualized)')
plt.grid(True, alpha=0.3)
plt.show()

---
## Step 4 | Statistical Analysis

In [ ]:
# Model 1: Simple regression
X_simple = sm.add_constant(data['Minimum_Wage'])
model_simple = sm.OLS(data['Voter_Turnout'], X_simple).fit()
print("Model 1: Voter_Turnout ~ Minimum_Wage")
print(model_simple.summary().tables[1])

In [ ]:
# Model 2: Multiple regression
X_multi = data[['Minimum_Wage', 'Median_Household_Income']]
X_multi = sm.add_constant(X_multi)
model_multi = sm.OLS(data['Voter_Turnout'], X_multi).fit()
print("\nModel 2: Voter_Turnout ~ Minimum_Wage + Median_Household_Income")
print(model_multi.summary())

In [ ]:
# Key results
print("\n" + "="*50)
print("KEY RESULTS")
print("="*50)
print(f"\nNull Hypothesis: Minimum wage does not predict voter turnout (beta = 0)")
print(f"\nSimple Regression:")
print(f"  Minimum wage coefficient: {model_simple.params['Minimum_Wage']:.4f}")
print(f"  P-value: {model_simple.pvalues['Minimum_Wage']:.3f}")
print(f"\nMultiple Regression (with income control):")
print(f"  Minimum wage coefficient: {model_multi.params['Minimum_Wage']:.4f}")
print(f"  P-value: {model_multi.pvalues['Minimum_Wage']:.3f}")
print(f"  R-squared: {model_multi.rsquared:.3f}")
print(f"\nConclusion: FAIL TO REJECT null hypothesis")
print(f"  Minimum wage is NOT significantly associated with voter turnout")

---
## Step 5 | Results Interpretation

### Key Findings

| Model | Min Wage Coef | P-value | Significant? |
|-------|--------------|---------|-------------|
| Simple | -0.08 | 0.52 | No |
| With income | 0.15 | 0.38 | No |

**R-squared:** 0.008 (essentially zero)

1. **No Relationship:** Minimum wage does not predict voter turnout

2. **Coefficient Changes Sign:** Simple regression shows slight negative effect, multiple regression shows slight positive - neither significant

3. **Very Low R²:** Minimum wage explains less than 1% of turnout variation

### Why No Relationship?

- **Different drivers:** Voter turnout is driven by election competitiveness, candidate quality, and mobilization
- **Policy salience:** Minimum wage may not be a primary voting motivation
- **Aggregation:** State-level analysis may miss individual-level effects
- **Confounders:** Many other factors affect both variables

### This is a Null Result

Null results are important! This suggests that:
- Minimum wage policy is not obviously linked to voter engagement
- Other factors are much more important for turnout

---
## Replication Exercises

### Exercise 1: Year Fixed Effects
Add year dummies. Does the relationship change?

### Exercise 2: State Fixed Effects
Compare within-state changes. Do states that raise minimum wage see turnout changes?

### Exercise 3: Ballot Initiatives
In years with minimum wage ballot initiatives, is turnout higher?

### Challenge Exercise
Research the political economy of minimum wage. Who votes for minimum wage increases?

In [ ]:
# Your code for exercises

# Example: Turnout by year
# print(data.groupby('Year')['Voter_Turnout'].mean())